In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint


In [2]:
# Retrieve data from 2016 Electoral Votes CSV file
electoral_path = "C:/Users/palan/NW-Data-Science/Project-1/Electoral Votes/2016 Election - Electoral Votes.csv"
electoral_df = pd.read_csv(electoral_path)

# add year column for later merging
electoral_df['Year'] = 2016

electoral_df.head(n=5)


,State ID,State,Electoral Votes,Vote Weight,Turnout,Year
0,1,Wyoming,3,2.97,59.40%,2016
1,2,District of Columbia,3,2.45,60.20%,2016
2,3,Vermont,3,2.42,63.50%,2016
3,4,Alaska,3,2.39,61.30%,2016
4,5,Hawaii,4,2.37,41.70%,2016


In [3]:
# Retrieve data from Presidental Election CSV file
election_path = "C:/Users/palan/NW-Data-Science/Project-1/Electoral Votes/1976-2016-president.csv"
election_df = pd.read_csv(election_path)

# update data frame with only states, year, and total votes
total_votes_df = election_df[['year','state','totalvotes']]
total_votes_df = total_votes_df.drop_duplicates()

total_votes_df.head(n = 5)

,year,state,totalvotes
0,1976,Alabama,1182850
7,1976,Alaska,123574
11,1976,Arizona,742719
19,1976,Arkansas,767535
23,1976,California,7803770


In [4]:
# merge data frames: electoral votes with total votes by states
merged_df = electoral_df.merge(total_votes_df, how='inner', left_on=["Year", "State"], right_on=["year","state"])

# drop redundant state column
merged_df = merged_df.drop(columns=['state', 'year', 'Vote Weight'])

# initialize list for converting turnout percent to float
turnout_list = []

# loop through rows in turnout column, remove % sign, and convert string to float
for items in merged_df['Turnout']:
    
    new_percent = float(items[:-1]) / 100
    turnout_list.append(new_percent)

# convert list to series
turnout_series = pd.Series(turnout_list, name='percent', index=merged_df.index)
merged_df['Turnout'].update(turnout_series)

# add column: total eligible voting population
merged_df['Eligible Voters'] = merged_df['totalvotes'] / merged_df['Turnout']

# set baseline electoral vote share per regular vote for entire US (2016 election)
total_votes_2016 = merged_df['totalvotes'].sum()
total_electoral_2016 = merged_df['Electoral Votes'].sum()
baseline_worth = total_electoral_2016 / total_votes_2016
print(f'\nBaseline worth of a vote (Total Electoral Votes / Total Popular Vote) in 2016: {baseline_worth}\n')

merged_df.head(n=5)



Baseline worth of a vote (Total Electoral Votes / Total Popular Vote) in 2016: 3.933116922712944e-06



,State ID,State,Electoral Votes,Turnout,Year,totalvotes,Eligible Voters
0,1,Wyoming,3,0.594,2016,258788,435670
1,2,District of Columbia,3,0.602,2016,312575,519228
2,3,Vermont,3,0.635,2016,320467,504672
3,4,Alaska,3,0.613,2016,318608,519752
4,5,Hawaii,4,0.417,2016,437664,1.04955e+06


In [5]:
# calculate vote worth by state & normalize to baseline worth across country
merged_df['Actual Vote Worth'] = (merged_df['Electoral Votes'] / merged_df['totalvotes']) / baseline_worth

# calculate vote worth by eligible voters
merged_df['Elg. Vote Worth'] = (merged_df['Electoral Votes'] / merged_df['Eligible Voters']) / baseline_worth

merged_df.head(n=5)

,State ID,State,Electoral Votes,Turnout,Year,totalvotes,Eligible Voters,Actual Vote Worth,Elg. Vote Worth
0,1,Wyoming,3,0.594,2016,258788,435670,2.947408,1.75076
1,2,District of Columbia,3,0.602,2016,312575,519228,2.440227,1.46902
2,3,Vermont,3,0.635,2016,320467,504672,2.380132,1.51138
3,4,Alaska,3,0.613,2016,318608,519752,2.394020,1.46753
4,5,Hawaii,4,0.417,2016,437664,1.04955e+06,2.323712,0.968988


In [11]:
# Retrieve data from Presidental Election CSV file
geodata_path = "C:/Users/palan/NW-Data-Science/Project-1/Electoral Votes/states lat & long data.csv"
geodata_df = pd.read_csv(geodata_path)

geodata_df.head(n=5)

# merge merged_df with geographic data
new_merged_df = merged_df.merge(geodata_df, how='inner', left_on=["State"], right_on=["name"])

# drop redundant columns
new_merged_df = new_merged_df.drop(columns=['state', 'name'])

new_merged_df

,State ID,State,Electoral Votes,Turnout,Year,totalvotes,Eligible Voters,Actual Vote Worth,Elg. Vote Worth,latitude,longitude
0,1,Wyoming,3,0.594,2016,258788,435670,2.947408,1.75076,43.075968,-107.290284
1,2,District of Columbia,3,0.602,2016,312575,519228,2.440227,1.46902,38.905985,-77.033418
2,3,Vermont,3,0.635,2016,320467,504672,2.380132,1.51138,44.558803,-72.577841
3,4,Alaska,3,0.613,2016,318608,519752,2.394020,1.46753,63.588753,-154.493062
4,5,Hawaii,4,0.417,2016,437664,1.04955e+06,2.323712,0.968988,19.898682,-155.665857
5,6,North Dakota,3,0.591,2016,344360,582673,2.214990,1.30906,47.551493,-101.002012
6,7,Rhode Island,4,0.59,2016,464144,786685,2.191141,1.29277,41.580095,-71.477429
7,8,South Dakota,3,0.585,2016,370093,632638,2.060979,1.20567,43.969515,-99.901813
8,9,West Virginia,5,0.499,2016,713051,1.42896e+06,1.782841,0.889638,38.597626,-80.454903
9,10,Delaware,3,0.637,2016,441590,693234,1.727290,1.10028,38.910832,-75.527670


In [15]:
import gmaps
import os

# set gmaps api key
api_txt = "C:/Users/palan/NW-Data-Science/Google Maps API Key.txt"
txt = open(api_txt,"r")

with txt:
    
    gkey = txt.read()

# configuring api key
gmaps.configure(api_key = gkey)

# initiating map
fig = gmaps.figure()

# specifying maps figure parameters
figure_layout = {
    'width': '400 px',
    'width': '300 px',
    'border': '1px solid black',
    'padding': '1px'
                    }


fig = gmaps.figure(layout = figure_layout)

# specifying location & weight values for heat map layer
states = new_merged_df[["latitude", "longitude"]].astype(float)
vote_weight = new_merged_df['Actual Vote Worth'].astype(float) * 50

heat_layer = gmaps.heatmap_layer(states, weights = vote_weight, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 2)

# adding heat map layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='1px', width='300 px'))